# Efficient approximation of Brownian Lévy area

We shall present a new (weak) approximation for the Lévy area between two independent Brownian motions $W^{(1)}$ and $W^{(2)}$,

\begin{align}
A_{s,t} := \frac{1}{2}\left(\int_s^t W_{s,u}^{(1)} dW_u^{(2)} - \int_s^t W_{s,u}^{(2)} dW_u^{(1)}\right).
\end{align}

It was shown in [1] that by replacing Lévy area increments with Gaussian approximations that have the same covariance structure, one can develop high order (in a 2-Wasserstein sense) numerical methods for SDEs driven by multidimensional Brownian motion.

Inspired by this approach, we will consider an approximation that matches additional (conditional) moments of two-dimensional Lévy area.

Firstly, we define the space-time Lévy area of a Brownian motion $W$ over the interval $[s,t]$ as

\begin{align}
H_{s,t} := \frac{1}{h}\int_s^t W_{s,u}-\frac{u-s}{h}W_{s,t}\,du,
\end{align}

where $h := t - s$. It is straightforward to show that $H_{s,t}$ has a $\textit{Normal}\big(0,\frac{1}{12}h\big)$ distribution and is independent of $W_{s,t}$ (see [2] for a proof).

Then the proposed approximation is defined as

\begin{align}
\tilde{A}_{s,t} := H_{s,t}^{(1)}W_{s,t}^{(2)} - W_{s,t}^{(1)}H_{s,t}^{(2)} + \sigma_{s,t}\lambda_{s,t},
\end{align}

where conditional on $ H_{s,t} $, the constant $\sigma_{s,t}$ is given
\begin{align}
\sigma_{s,t}^{2} & := \frac{1}{20}h^{2} + \frac{1}{5}h\left(\left(H_{s,t}^{(1)}\right)^{2} + \left(H_{s,t}^{(2)}\right)^{2}\right),
\end{align}

and the random variable $\lambda_{s,t}$ is generated entirely from independent random variables as

\begin{align}
\lambda_{s,t} & \sim \begin{cases}
\textit{Logistic}\big(0, \frac{\sqrt{3}}{\pi}\big) & \text{with probability}\,\,\,p_{s,t}\\[3pt]
\textit{Normal}\big(0, 1\big)& \text{with probability}\,\,\,1-p_{s,t}\\
\end{cases},\\[3pt]
p_{s,t} & := \frac{1}{\sigma_{s,t}^{4}}\left(\frac{1}{560}h^{4}+\frac{1}{70}h^{3}\left(\left(H_{s,t}^{\left(1\right)}\right)^{2}+\left(H_{s,t}^{\left(2\right)}\right)^{2}\right)\right).
\end{align}

The random variable $\tilde{A}_{s,t}$ is designed so that

\begin{align}
\mathbb{E}\left[\left(\tilde{A}_{s,t}\right)^{k} \,\big|\,\, W_{s,t}, H_{s,t}\right] = \mathbb{E}\left[\left(A_{s,t}\right)^{k} \,\big|\,\, W_{s,t}, H_{s,t}\right],
\end{align}

for $k \leq 5$. (The proof of this will be detailed in my PhD thesis)

## Quantifying the approximation error

Recall that the 2-Wasserstein distance between two probability measures $\mu$ and $\nu$ is given by 

\begin{align}
W_2(\mu, \nu) := \Big(\inf_{X\sim \mu,\, Y\sim \nu} \mathbb{E}\big[\left|X-Y\right|^2\big]\Big)^\frac{1}{2},
\end{align}

where the infimum is taken over all joint distributions of $X$ and $Y$ with marginals $\mu$ and $\nu$ respectively.

We shall numerically estimate the 2-Wasserstein distance between the (conditional) distributions of $\tilde{A}_{s,t}$ and $A_{s,t}$.

This can be done using the estimator studied in [3] which is computable using the following three steps:

$\textbf{Step 1.}$ Generate $N$ independent samples from each distribution.

$\textbf{Step 2.}$ Couple these samples via their order statistics (i.e. sort both samples and identify the k-th elements in each list).

$\textbf{Step 3.}$ Compute the L2 error using this coupling.

The hope is that this estimator will then approach the true 2-Wasserstein error as $N$ increases, i.e.

\begin{align}
\frac{1}{N}\sum_{i=1}^N |A_{(i)} - \tilde{A}_{(i)}|^2 \rightarrow W_2^2(\mu_{A}, \tilde{\mu}_{A})\hspace{4mm}\text{almost surely as}\hspace{2mm}N\rightarrow \infty,
\end{align}

where $\mu_{A}$ and $\tilde{\mu}_{A}$ denote the distributions of $A_{s,t}$ and $\tilde{A}_{s,t}$ conditional on $W_{s,t}$.

## Generating an approximate Lévy area

In [1]:
import math
import numpy as np

# Precomputed constants
root_twelveth = math.sqrt(1.0/12.0)
root_three_over_pi = math.sqrt(3.0)/math.pi
constant1 = 1.0/560.0
constant2 = 1.0/70.0

# Generates a sample of tilde{A}_{s,t} conditional on the increments of W over [s,t]
def weak_levy_area(h, sqrt_h, increment1, increment2):
    
    area_stddev = root_twelveth*sqrt_h
    
    # Generate the space-time Levy areas
    area1 = np.random.normal(0.0, area_stddev)
    area2 = np.random.normal(0.0, area_stddev)
    
    area_size = (area1 ** 2.0) +  (area2 ** 2.0)
    
    variance = 0.05 * h + 0.2 * area_size
    stddev = math.sqrt(variance*h)
    
    # Compute the probability p_{s,t}
    p = 1.0 - (constant1 * (h ** 2.0) + constant2 * h * area_size)/(variance ** 2.0)
        
    bridge_area = 0.0

    # Generate lambda_{s,t}
    if np.random.binomial(1, p) == 1:
        bridge_area = np.random.normal(0.0, stddev)
    else:
        bridge_area = np.random.logistic() * root_three_over_pi * stddev
    
    return  area1 * increment2 - increment1 * area2 + bridge_area

## Discretization of Brownian Lévy area

To discretize $A_{s,t}$ conditional on $W_{s,t}$, we will use the following two facts:


$\textbf{Fact 1.}$ By the additivity of integration, we have
\begin{align}
A_{s,t} = A_{s,u} + \frac{1}{2}\big(W_{s,u}^{(1)}W_{u,t}^{(2)} - W_{s,u}^{(2)}W_{u,t}^{(1)}\big) + A_{u,t},
\end{align}
where $u := s + \frac{1}{2}h$ is the midpoint of $[s,t]$.

$\textbf{Fact 2.}$ Lévy area is a centered random variable with
\begin{align}
\mathbb{E}\big[\,A_{s,t}\left|\, W_{s,t}\right.\big] = 0.
\end{align}

In [2]:
# Precomputed constant
root_halve = math.sqrt(0.5)

# Generates an approximate sample of A_{s,t} conditional on the increment W_{s,t}.
# The step size used to discretize A_{s,t} is h * 2^(-refine_counter).
def actual_levy_area(sqrt_h, increment1, increment2, refine_counter):
    
    #  This is the base case of the recursion
    if refine_counter == 0:
        return 0.0
    
    # Generate the Brownian motion at the midpoint of the interval
    # conditional on W_{s,t} using the Brownian bridge at u = s + 0.5h.
    first_w1 = 0.5 * (increment1 + np.random.normal(0.0, sqrt_h))
    first_w2 = 0.5 * (increment2 + np.random.normal(0.0, sqrt_h))
    
    second_w1 = increment1 - first_w1
    second_w2 = increment2 - first_w2
    
    sqrt_half_h = root_halve * sqrt_h
    
    # Compute A_{s,t} recursively (using the two half intervals)
    return 0.5 * (first_w1 * second_w2 - first_w2 * second_w1) \
            + actual_levy_area(sqrt_half_h, first_w1, first_w2, refine_counter - 1) \
            + actual_levy_area(sqrt_half_h, second_w1, second_w2, refine_counter - 1)

## Estimating the 2-Wasserstein distance

In [3]:
# Estimates the 2-Wasserstein distance between two sets of N samples
def wasserstein2(sample1, sample2):
    sample1.sort()
    sample2.sort()
    
    n = len(sample1)
    error = 0
    
    for i in range(n):
        error = error + (sample1[i] - sample2[i]) ** 2.0
        
    return error / n

## Running a numerical experiment

In [5]:
# Size of the interval h = t - s
h = 1.0
sqrt_h = math.sqrt(h)

# Fix the increments of the Brownian motion over the interval [s,t]
increment1 = 1.0 * sqrt_h
increment2 = -0.5 * sqrt_h

# Number of samples
n = 500000

weak_levy_areas = []
actual_levy_areas = []

# Generate independent samples from each distribution conditional on W_{s,t}
for i in range(n):
    weak_levy_areas.append(weak_levy_area(h, sqrt_h, increment1, increment2))
    actual_levy_areas.append(actual_levy_area(sqrt_h, increment1, increment2, 7))

# Estimate the 2-Wasserstein error from the samples
print('Estimated 2-Wasserstein error is', 
      math.sqrt(wasserstein2(weak_levy_areas, actual_levy_areas)))

Estimated 2-Wasserstein error is 0.002602999521509086


## Conclusion

Empircal evidence suggests that the 2-Wasserstein error for the proposed approximation of Lévy area is small:

\begin{align}
W_{2}(\mu_{A}, \tilde{\mu}_{A}) \leq 0.01h.
\end{align}

## References

[1] G. Flint and T. Lyons, Pathwise approximation of sdes by coupling piecewise abelian rough
paths, https://arxiv.org/abs/1505.01298, 2015.

[2] J. Foster, T. Lyons and H. Oberhauser, An optimal polynomial approximation of Brownian motion, https://arxiv.org/abs/1904.06998, 2019.

[3] T. Klein, J. Fort, P. Berthet, Convergence of an estimator of the Wasserstein
distance between two continuous probability distributions, hal-01526879, 2017.